### Regression as machine learing

In [ ]:
!pip install ase
!git clone https://github.com/icomse/5th_workshop_MachineLearning.git
import os
os.chdir('5th_workshop_MachineLearning/data')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
# the names in sklearn tend to be pretty long, so we will import the individual
# objects rather than the whole thing

### Let's read in some data

This is some artificial data we will be playing around with to illustrate some important concepts.

In [ ]:
dlin = pd.read_csv('linmod.csv')

What does this data look like? Inspect directly and plot.

In [ ]:
dlin.head()

In [ ]:
dlin.plot(x='inputs',y='outputs',kind='scatter')
plt.show()

Now, let's train a linear model with the data! First, create the model. It starts out as just an empty object, that we need to fill.

In [ ]:
linmodel = LinearRegression()

Now look up the documentation as to what to do to fit the model. When you fit it, what do you get back? NOTE: `LinearRegression` expects that that the input X is a 2D arrray, of size `[n_features, n_samples]`, because it is designed to handle the general case that you are doing multiple variable linear regression. So if you pass in a 1D array, it will error. Read the error message you get carefully for a quick fix.

In [ ]:
X = dlin['inputs'].values.reshape(-1, 1)
Y = dlin['outputs']
linmodel.fit(X,Y)

OK, you have a model. We next want to plot the prediction of the model and the points on the same plot?

In [ ]:
xvals = np.linspace(10,70,100)
ypred = linmodel.predict(xvals.reshape(-1,1))  # use this to apply the model
plt.plot(xvals,ypred,c='k')
plt.scatter(X,Y)
plt.xlabel('inputs')
plt.ylabel('outputs')
plt.show()

What methods does the `LinearRegression` have? What information can you get from it?  What are the parameters of a 1D linear regression?  What is a measure of how good the model is?

In [ ]:
linmodel?

In [ ]:
print(linmodel.intercept_)
print(linmodel.coef_)
print(linmodel.score(X,Y))  # in this case, the score gives the R^2.

Let's play around with (over)fitting!  There's a little bit of curvature - in the data; maybe we should try a polynomial fit?

Let's actually just use a sample of the data set to train; we want to go up to the number of data points, and polynomial regression is numerically unstable with large numbers.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

We make a polynomial fit using sklearn by taking powers of the input features.  `scikit-learn` has a function for this.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

To use it, you create a `PolynomialFeatures` object of the degree you are interested in, then take the powers of the data using the fit_transform method of `PolynomialFeatures`. See example (you will have to supply your own X, which is the same input as `LinearRegression.fit` takes.

In [ ]:
pf = PolynomialFeatures(degree=2)
pX_train = pf.fit_transform(X_train)

In [ ]:
print(X_train[:5])
print(pX_train[:5])

it's a degree $\times$ n_data array; with each column the input data to power $n$.

Now, lets see how the model fits the data.  Inspect the model parameters and the $R^2$, and plot the fit and the curve.

In [ ]:
linmodel.fit(pX_train,Y_train)
print(linmodel.intercept_)
print(linmodel.coef_)
print('R^2 (train) =', linmodel.score(pX_train,Y_train))

In [ ]:
xvals = np.linspace(10,70,100)
px = pf.fit_transform(xvals.reshape(-1,1))
ypredict = linmodel.predict(px)
plt.plot(xvals,ypredict,c='k')
plt.scatter(X_train,Y_train)
plt.xlabel('inputs')
plt.ylabel('outputs')
plt.show()

Now test the model on the test set.

In [ ]:
pX = pf.fit_transform(X_test)
plt.plot(xvals,ypredict,c='k')
plt.scatter(X_test,Y_test)
plt.xlabel('inputs')
plt.ylabel('outputs')
print('R^2 =', linmodel.score(pX,Y_test))

Remember, it's the TESTING MSE (or $R^2$) that we want to minimize, not the training MSE.  If the training MSE is low, but the testing is high, then the model is overfit.  

**HACKING TIME**: Now, go back and try a higher polynomial degree. Note that polynomial fitting is not very numerically stable, so if you fit to too many points, then you start to have problems.  What problems do you have?

Can you plot $R^2$ (i.e. linmodel.score) for the test and training set versus polynomial degree?  What do you see?

In [ ]:
pf = PolynomialFeatures(degree=4)
pX_train = pf.fit_transform(X_train) # transform the features into a x^0 . . . x^15
linmodel.fit(pX_train,Y_train) # train the model on the data
print('R^2 (train) =', linmodel.score(pX_train,Y_train))
pxvals = pf.fit_transform(xvals.reshape(-1,1)) # generate the polynomial input data
ypredict = linmodel.predict(pxvals) # predict the curve of the model.
plt.plot(xvals,ypredict,c='k')
plt.scatter(X_test,Y_test,label='test',alpha=0.5)
plt.scatter(X_train,Y_train,label='train',alpha=0.5)
plt.legend()
pX_test = pf.fit_transform(X_test)  # what is the transformed test data, so we can compute the score
print('R^2 (test) =', linmodel.score(pX_test,Y_test)) # how well does the test data fit?

In [ ]:
R2test = list()
R2train = list()
ps = list()
for p in range(2,20):
    pf = PolynomialFeatures(degree=p)
    pX_train = pf.fit_transform(X_train)
    linmodel.fit(pX_train,Y_train)
    R2train.append(linmodel.score(pX_train,Y_train))
    pxvals = pf.fit_transform(xvals.reshape(-1,1)) # generate the polynomial input data
    ypredict = linmodel.predict(pxvals)
    pX_test = pf.fit_transform(X_test)
    R2test.append(linmodel.score(pX_test,Y_test))
    ps.append(p)

In [ ]:
plt.plot(ps,R2test,label='test set')
plt.plot(ps,R2train,label='training set')
plt.xlabel('polynomial degree')
plt.ylabel('$R^2$')
plt.legend()
plt.show()

Generally the training set will get a bit better as the model gets more complex, whereas the test model gets worse as the trained model starts to overfit.